In [109]:
import lynx

In [110]:
lk = lynx.LynxKite(address='http://localhost:2200/')

In [111]:
import pandas

In [120]:
edges = lk.new_project().newvertexset(size=100) \
  .vertexattributetodouble(attr='ordinal') \
  .derivedvertexattribute(output='src', type='string', expr='ordinal % 44') \
  .derivedvertexattribute(output='dst', type='string', expr='ordinal % 77')
t = edges.sql('select * from vertices').to_table()
p = lk.new_project().importverticesandedgesfromasingletable(table=t.name, src='src', dst='dst')

pandas.DataFrame(p.findvertexcoloring(name='color').sql('select * from vertices').take(10))



,color,id,stringID
0,1.0,-9103770305901363147,72
1,1.0,-8499159135349112773,76
2,1.0,-8361175665923325896,17
3,1.0,-8292973307544731636,15
4,1.0,-8225913900093145057,16
5,2.0,-8062155292541976514,44
6,1.0,-7685764892959178689,70
7,1.0,-7423979210269523952,48
8,2.0,-7261648962812116986,57
9,1.0,-7138230367437193190,0


In [197]:
partition1 = 6
partition2 = 2

edges = lk.new_project() \
.derivescalar(output = 'partition1', type = 'double', expr = str(partition2)) \
.derivescalar(output = 'partition2', type = 'double', expr = str(partition1)) \
.newvertexset(size = partition1 * partition2) \
.vertexattributetodouble(attr = 'ordinal') \
.derivedvertexattribute(output = 'src', type = 'string', expr = 'Math.floor(ordinal / partition1)') \
.derivedvertexattribute(output = 'dst', type = 'string', expr = '(ordinal % partition1) + partition2')
t = edges.sql('select * from vertices').to_table()

graph = lk.new_project().importverticesandedgesfromasingletable(table = t.name, src = 'src', dst = 'dst') \
.derivescalar(output = 'partition2', type = 'double', expr = str(partition1)) \
.derivedvertexattribute(output = 'truePartition', type = 'double', expr = 'stringID < partition2 ? 1.0 : -1.0') \
.addgaussianvertexattribute(name = 'a', seed = '13') \
.derivedvertexattribute(output = 'partition', type = 'double', expr = 'a < -0.7 ? undefined : truePartition') \
.trainneuralnetworkanduseittopredictanattribute( \
    label = "partition", \
    output = "prediction", \
    features = "!unset",
    networkSize = "4", \
    learningRate = "0.02", \
    radius = "3", \
    hideState = "false", \
    forgetFraction = "0.3", \
    trainingRadius = "1", \
    maxTrainingVertices = "8", \
    minTrainingVertices = "7", \
    iterationsInTraining = "50", \
    subgraphsInTraining = "10", \
    knownLabelWeight = "0.5", \
    numberOfTrainings = "10")

pandas.DataFrame(graph.sql('select * from vertices').take(100))

,a,id,partition,prediction,stringID,truePartition
0,1.682883,-7261648962812116986,1.0,0.997835,2,1.0
1,-0.405603,-4964420949324595199,-1.0,-0.998616,7,-1.0
2,-0.037654,-4962768464952426496,1.0,0.997835,4,1.0
3,0.018468,-3109364766762598398,-1.0,-0.998616,6,-1.0
4,-0.164347,1853403698189828099,1.0,0.997835,5,1.0
5,-0.767720,4437113782736519172,NaN,0.997823,0,1.0
6,0.822000,7515937758611767303,1.0,0.997835,1,1.0
7,-0.202210,7564655869003038725,1.0,0.997835,3,1.0
